# Companion Notebook -- Kerem Uysal

In [56]:
import pandas as pd
import altair as alt
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import numpy as np
import plotly.express as px
from shapely.geometry import Point
import plotly.graph_objs as go
import plotly.subplots as tls

# The Data

# Dataset Overview
The dataset used in this project is a historical compilation of the modern Olympic Games, spanning from the first Games in Athens in 1896 to the 2016 Games in Rio de Janeiro. This dataset captures the essence of the Olympic competitions through the years, detailing the events, athletes, results, and medal distributions.
# Source
The data was originally scraped from sports-reference.com in May 2018. The comprehensive scraping and data wrangling efforts were undertaken by Kaggle user **RGRIFFIN**, whose R scripts and methodologies are available on GitHub. The dataset can be accessed directly on Kaggle with the link below.
https://www.kaggle.com/datasets/heesoo37/120-years-of-olympic-history-athletes-and-results
## 1. Athlete Events Dataset (`athlete_events.csv`)


**Features:**

- **ID**: Unique number for each athlete
- **Name**: Athlete's name
- **Sex**: Gender of the athlete (M or F)
- **Age**: Age of the athlete at the time of competition
- **Height**: Height of the athlete in centimeters
- **Weight**: Weight of the athlete in kilograms
- **Team**: Name of the team the athlete represents (can be a country or a region)
- **NOC**: National Olympic Committee 3-letter code
- **Games**: The year and season of the Olympics
- **Year**: Year of the Olympics
- **Season**: Season of the Olympics (Summer or Winter)
- **City**: Host city of the Olympics
- **Sport**: Sport in which the athlete competed
- **Event**: Specific event in which the athlete competed
- **Medal**: Type of medal won (Gold, Silver, Bronze, or NA if no medal was won)

## 2. NOC Regions Dataset (`noc_regions.csv`)

This dataset provides information about the National Olympic Committees, including a mapping of the NOC codes to the corresponding country or region names, along with additional regional information.

**Features:**

- **NOC**: National Olympic Committee 3-letter code
- **region**: Corresponding geographical region or country
- **notes**: Additional notes or historical changes concerning the region or NOC code

### Relevance to Project
The dataset is really important for our project because it gives a detailed look at how female athletes have taken part and performed in the Olympics over the years. By studying this data, we can learn about trends, see how much progress has been made in gender equality in sports, and understand how different countries have supported their female athletes in the Olympic Games.

In [57]:
# Load the datasets
athletes = pd.read_csv('athlete_events.csv')
regions = pd.read_csv('noc_regions.csv')

In [58]:
athletes.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN


In [59]:
regions.head()

,NOC,region,notes
0,AFG,Afghanistan,NaN
1,AHO,Curacao,Netherlands Antilles
2,ALB,Albania,NaN
3,ALG,Algeria,NaN
4,AND,Andorra,NaN


### Merging the Datasets

In [60]:
# Merging the datasets on the 'NOC' column
merged_data = pd.merge(athletes, regions, on='NOC', how='left')

In [61]:
merged_data.head()

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region,notes
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,China,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,China,NaN
2,3,Gunnar Nielsen Aaby,M,24.0,NaN,NaN,Denmark,DEN,1920 Summer,1920,Summer,Antwerpen,Football,Football Men's Football,NaN,Denmark,NaN
3,4,Edgar Lindenau Aabye,M,34.0,NaN,NaN,Denmark/Sweden,DEN,1900 Summer,1900,Summer,Paris,Tug-Of-War,Tug-Of-War Men's Tug-Of-War,Gold,Denmark,NaN
4,5,Christine Jacoba Aaftink,F,21.0,185.0,82.0,Netherlands,NED,1988 Winter,1988,Winter,Calgary,Speed Skating,Speed Skating Women's 500 metres,NaN,Netherlands,NaN


In [62]:
# Calculate the percentage of missing values for each column
missing_percentages = (merged_data.isnull().sum() / len(merged_data)) * 100

# Display the percentages in a more readable format
print(missing_percentages.sort_values(ascending=False))


notes     98.141386
Medal     85.326207
Weight    23.191180
Height    22.193821
Age        3.494445
region     0.136473
Season     0.000000
Event      0.000000
Sport      0.000000
City       0.000000
ID         0.000000
Year       0.000000
Name       0.000000
NOC        0.000000
Team       0.000000
Sex        0.000000
Games      0.000000
dtype: float64


## Missing Data Analysis

After merging the datasets and calculating the percentage of missing values for each column, I have found several issues.

- **Notes**: Missing in 98.14% of entries. Almost most of the column is mising. Therefore I decided to drop it out.

- **Medal**: 85.32% missing. This high rate represents the athletes who  did not win a medal. I am going to fill these as "None".
- **Weight**: 23.19% missing. This is a significant portion of the data.  I am going to impute these missing values using the  mean weight grouped by sport can help maintain data integrity.

- **Height**: 22.39% missing. Similar to weight, a substantial amount of height data is missing. I will impute these values using the  mean height by sport , as physical attributes will be significantly different among different sports.

- **Age**: 3.19% missing. A smaller portion of the age data is missing. I am going to impute this column as median as it is only 3% of data. 

- **Region**: 0.13% missing. This is a very small percentage. Which is not that important but I will check the NOC codes and fill these regions.

## Imputing Missing values

- I am going to start imputing with regions.

In [63]:
missing_regions = merged_data[merged_data['region'].isna()]


In [64]:
unique_nocs = missing_regions['NOC'].unique()
unique_nocs

array(['SGP', 'ROT', 'UNK', 'TUV'], dtype=object)

In [65]:
unk_rows = merged_data[merged_data['NOC'] == 'UNK']

# Display the rows with 'UNK' as the region
unk_rows

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal,region,notes
61080,31292,Fritz Eccard,M,NaN,NaN,NaN,Unknown,UNK,1912 Summer,1912,Summer,Stockholm,Art Competitions,Art Competitions Mixed Architecture,NaN,NaN,Unknown
130721,65813,A. Laffen,M,NaN,NaN,NaN,Unknown,UNK,1912 Summer,1912,Summer,Stockholm,Art Competitions,Art Competitions Mixed Architecture,NaN,NaN,Unknown


In [66]:
region_updates = {
    'SGP': 'Singapore',   
    'ROT': 'Refugee Olympic Team',
    'UNK': 'Unknown',
    'TUV': 'Tuvalu'       
}

# Apply the updates to the 'region' column based on NOC codes
for noc, region in region_updates.items():
    merged_data.loc[merged_data['NOC'] == noc, 'region'] = region

# Re-check if there are still missing values in the 'region' column
print((merged_data['region'].isnull().sum() / len(merged_data)) * 100)

0.0


In [67]:
# 1. Drop the 'Notes' column
merged_data.drop(columns='notes', inplace=True)

# 2. Fill missing 'Medal' values with 'None'
merged_data['Medal'].fillna('None', inplace=True)

# 3. Impute missing 'Weight' using the mean weight grouped by 'Sport' and 
merged_data['Weight'] = merged_data.groupby('Sport')['Weight'].transform(lambda x: x.fillna(x.mean()))

# 4. Impute missing 'Height' using the mean height grouped by 'Sport' and 
merged_data['Height'] = merged_data.groupby('Sport')['Height'].transform(lambda x: x.fillna(x.mean()))

# 5. Impute missing 'Age' using the median age
merged_data['Age'].fillna(merged_data['Age'].median(), inplace=True)

In [68]:
print(merged_data.isnull().sum())

ID          0
Name        0
Sex         0
Age         0
Height     99
Weight    217
Team        0
NOC         0
Games       0
Year        0
Season      0
City        0
Sport       0
Event       0
Medal       0
region      0
dtype: int64


It's possible that some sports have all their Height or Weight data missing. If any of the athletes in a particular sport have recorded weights or heights, the mean calculation will return Nan. Lets check if it is true or not.

In [69]:
# Count non-missing 'Height' and 'Weight' values for each sport
height_counts = merged_data.groupby('Sport')['Height'].count().reset_index(name='Height Non-Missing Count')
weight_counts = merged_data.groupby('Sport')['Weight'].count().reset_index(name='Weight Non-Missing Count')

# Merge the counts back for comparison
counts_per_sport = pd.merge(height_counts, weight_counts, on='Sport')

In [70]:
# Filter to find sports with no non-missing 'Height' or 'Weight' data
no_height_data_sports = counts_per_sport[counts_per_sport['Height Non-Missing Count'] == 0]
no_weight_data_sports = counts_per_sport[counts_per_sport['Weight Non-Missing Count'] == 0]

# Display the sports with no recorded 'Height' or 'Weight'
print("Sports with no recorded Height data:", no_height_data_sports['Sport'].tolist())
print("Sports with no recorded Weight data:", no_weight_data_sports['Sport'].tolist())

Sports with no recorded Height data: ['Aeronautics', 'Alpinism', 'Basque Pelota', 'Cricket', 'Croquet', 'Military Ski Patrol', 'Roque']
Sports with no recorded Weight data: ['Aeronautics', 'Alpinism', 'Basque Pelota', 'Cricket', 'Croquet', 'Jeu De Paume', 'Military Ski Patrol', 'Polo', 'Racquets', 'Roque']


There are several sports that does not have any weight and height data. I am going to impute these as general median of the height and weight.

In [71]:

if merged_data['Height'].isnull().any():
    overall_median_height = merged_data['Height'].median()
    merged_data['Height'].fillna(overall_median_height, inplace=True)

if merged_data['Weight'].isnull().any():
    overall_median_weight = merged_data['Weight'].median()
    merged_data['Weight'].fillna(overall_median_weight, inplace=True)

In [72]:
print(merged_data.isnull().sum())

ID        0
Name      0
Sex       0
Age       0
Height    0
Weight    0
Team      0
NOC       0
Games     0
Year      0
Season    0
City      0
Sport     0
Event     0
Medal     0
region    0
dtype: int64


- All of the missing values are now imputed.

## Statistics

In [73]:
merged_data.describe()


,ID,Age,Height,Weight,Year
count,271116.000000,271116.000000,271116.000000,271116.000000,271116.000000
mean,68248.954396,25.502493,175.132829,70.606476,1978.378480
std,39022.286345,6.287361,9.684578,13.063048,29.877632
min,1.000000,10.000000,127.000000,25.000000,1896.000000
25%,34643.000000,22.000000,169.000000,62.000000,1960.000000
50%,68205.000000,24.000000,175.000000,70.000000,1988.000000
75%,102097.250000,28.000000,181.000000,77.000000,2002.000000
max,135571.000000,97.000000,226.000000,214.000000,2016.000000


In [74]:
females = merged_data[merged_data['Sex'] == 'F']

In [75]:
female_stats = females[['Age', 'Height', 'Weight', 'Year']].describe()
print(female_stats)

                Age        Height        Weight          Year
count  74522.000000  74522.000000  74522.000000  74522.000000
mean      23.734401    168.309158     60.776296   1992.445748
std        5.778777      8.678878     10.174889     20.458993
min       11.000000    127.000000     25.000000   1900.000000
25%       20.000000    162.936020     55.000000   1984.000000
50%       23.000000    168.000000     60.000000   1998.000000
75%       27.000000    174.000000     67.000000   2008.000000
max       74.000000    213.000000    167.000000   2016.000000


- **Age and General Statistics**: Female athletes have an average age of 23.77 years, with ages ranging from 11 to 74 years.
- **Physical Dimensions**: The average height and weight are 168.37 cm and 60.77 kg respectively, with heights ranging from 127 to 213 cm and weights from 25 to 167 kg.
- **Participation Year**: The data covers Olympic participation from 1900 to 2016, with a median year of 1998.



In [76]:
# Medal distribution for female athletes
female_medal_distribution = females['Medal'].value_counts()
print("Female Medal Distribution:\n", female_medal_distribution)

# Number of different sports and events participated by females
female_sports_count = females['Sport'].nunique()
female_events_count = females['Event'].nunique()
print("Number of different sports by females:", female_sports_count)
print("Number of different events by females:", female_events_count)

Female Medal Distribution:
 Medal
None      63269
Bronze     3771
Gold       3747
Silver     3735
Name: count, dtype: int64
Number of different sports by females: 53
Number of different events by females: 269


### References

I used several resources to help with this project:

- **AI Assistance**: I used OpenAI's ChatGPT to get ideas, solve problems, and improve my project. It helped me understand my data better and make my project stronger.
- **Course Materials**: The lectures and materials from our class taught me how to manage and show my data. They were very important for doing my analysis.
- **Online Resources**: I also looked at different websites and articles. They gave me more information about data science and sports, which helped me learn more about the Olympics and gender equallympics.


In [55]:
merged_data.to_csv('merged_data.csv', index=False)